In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn
from sklearn.model_selection import train_test_split

C:\Users\Piyush\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_table('all/labeledTrainData.tsv')

In [3]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
train.shape

(25000, 3)

In [5]:
print(train.review[0])

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally star

In [6]:
train['review_bs']=train['review'].apply(lambda x:BeautifulSoup(x,'html.parser'))

In [7]:
print(train.review_bs[0])

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br/><br/>Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br/><br/>The actual feature film bit when it finally starts i

In [8]:
train.review_bs[0].get_text()

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [9]:
#again cleaning the data 
train['review_letters']=train['review_bs'].apply(lambda x:re.sub(r'[^a-zA-Z]',' ',x.get_text()))

In [10]:
train.review_letters[0]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [11]:
train.head()

,id,sentiment,review,review_bs,review_letters
0,5814_8,1,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\The Classic War of the Worlds\"" by Timothy Hi...",The Classic War of the Worlds by Timothy Hi...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,The film starts with a manager (Nicholas Bell)...,The film starts with a manager Nicholas Bell ...
3,3630_4,0,It must be assumed that those who praised this...,It must be assumed that those who praised this...,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious ...


In [12]:
train['review_words']=train['review_letters'].apply(lambda x:x.lower().split())

In [13]:
train.head()

,id,sentiment,review,review_bs,review_letters,review_words
0,5814_8,1,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...,"[with, all, this, stuff, going, down, at, the,..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\The Classic War of the Worlds\"" by Timothy Hi...",The Classic War of the Worlds by Timothy Hi...,"[the, classic, war, of, the, worlds, by, timot..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,The film starts with a manager (Nicholas Bell)...,The film starts with a manager Nicholas Bell ...,"[the, film, starts, with, a, manager, nicholas..."
3,3630_4,0,It must be assumed that those who praised this...,It must be assumed that those who praised this...,It must be assumed that those who praised this...,"[it, must, be, assumed, that, those, who, prai..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious ...,"[superbly, trashy, and, wondrously, unpretenti..."


In [14]:
set_of_stopwords =set(stopwords.words("english"))
train['review_meaningful_words']=train['review_words'].apply(lambda x:[w for w in x if not w in set_of_stopwords])

In [15]:
train.head()

,id,sentiment,review,review_bs,review_letters,review_words,review_meaningful_words
0,5814_8,1,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...,"[with, all, this, stuff, going, down, at, the,...","[stuff, going, moment, mj, started, listening,..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\The Classic War of the Worlds\"" by Timothy Hi...",The Classic War of the Worlds by Timothy Hi...,"[the, classic, war, of, the, worlds, by, timot...","[classic, war, worlds, timothy, hines, enterta..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,The film starts with a manager (Nicholas Bell)...,The film starts with a manager Nicholas Bell ...,"[the, film, starts, with, a, manager, nicholas...","[film, starts, manager, nicholas, bell, giving..."
3,3630_4,0,It must be assumed that those who praised this...,It must be assumed that those who praised this...,It must be assumed that those who praised this...,"[it, must, be, assumed, that, those, who, prai...","[must, assumed, praised, film, greatest, filme..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious ...,"[superbly, trashy, and, wondrously, unpretenti...","[superbly, trashy, wondrously, unpretentious, ..."


In [16]:
num_removed=len(train['review_words'][0]) - len(train['review_meaningful_words'][0])

In [17]:
num_removed

218

In [18]:
#for the first review entry  the number of  words removed are 218


In [19]:
#now making the words in a single string 
train['review_cleaned']=train['review_meaningful_words'].apply(lambda x: ' '.join(x))

In [20]:
train.review_cleaned[0]

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [21]:
train.head()

,id,sentiment,review,review_bs,review_letters,review_words,review_meaningful_words,review_cleaned
0,5814_8,1,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...,With all this stuff going down at the moment w...,"[with, all, this, stuff, going, down, at, the,...","[stuff, going, moment, mj, started, listening,...",stuff going moment mj started listening music ...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\The Classic War of the Worlds\"" by Timothy Hi...",The Classic War of the Worlds by Timothy Hi...,"[the, classic, war, of, the, worlds, by, timot...","[classic, war, worlds, timothy, hines, enterta...",classic war worlds timothy hines entertaining ...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,The film starts with a manager (Nicholas Bell)...,The film starts with a manager Nicholas Bell ...,"[the, film, starts, with, a, manager, nicholas...","[film, starts, manager, nicholas, bell, giving...",film starts manager nicholas bell giving welco...
3,3630_4,0,It must be assumed that those who praised this...,It must be assumed that those who praised this...,It must be assumed that those who praised this...,"[it, must, be, assumed, that, those, who, prai...","[must, assumed, praised, film, greatest, filme...",must assumed praised film greatest filmed oper...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious 8...,Superbly trashy and wondrously unpretentious ...,"[superbly, trashy, and, wondrously, unpretenti...","[superbly, trashy, wondrously, unpretentious, ...",superbly trashy wondrously unpretentious explo...


In [22]:
train.drop(['review','review_bs','review_letters','review_words','review_meaningful_words'],axis=1,inplace=True)

In [23]:
train.head()

,id,sentiment,review_cleaned
0,5814_8,1,stuff going moment mj started listening music ...
1,2381_9,1,classic war worlds timothy hines entertaining ...
2,7759_3,0,film starts manager nicholas bell giving welco...
3,3630_4,0,must assumed praised film greatest filmed oper...
4,9495_8,1,superbly trashy wondrously unpretentious explo...


In [24]:
#convert a collection of text documents to a matrix of token counts
#The CountVectorizer provides a simple way to both tokenize a 
#collection of text documents and build a vocabulary of known words,
#but also to encode new documents using that vocabulary.
#5000 indicates top 5000 frequent words in the bag thus linting the siZe of the feature vecture and speeding up the modeling process
#word indicates the words 
#processing none indicates that no processing is required by the data tokenising and removing stop words as our words in the bag 
vectorizer=CountVectorizer(analyzer="word",preprocessor=None,tokenizer=None,stop_words=None,max_features=5000)

In [25]:
train.review_cleaned[0]

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [26]:
train_data_features=vectorizer.fit_transform(list(train['review_cleaned'].values))

In [27]:
#converting the result into an array
train_data_features=train_data_features.toarray()




In [28]:
train_data_features[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [29]:
train_data_features.shape

(25000, 5000)

In [30]:
vocab=vectorizer.get_feature_names()# we can look at the bag of words which is a list of 5000 words

In [31]:
print(vocab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

In [32]:
test=pd.read_table('all/testData.tsv')

In [33]:
test

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...
5,2913_8,"...as valuable as King Tut's tomb! (OK, maybe ..."
6,4396_1,This has to be one of the biggest misfires eve...
7,395_2,"This is one of those movies I watched, and won..."
8,10616_1,The worst movie i've seen in years (and i've s...
9,9074_9,"Five medical students (Kevin Bacon, David Labr..."


In [34]:
test.shape

(25000, 2)

In [35]:
def clean_reviews(reviews,remove_stopwords=False,stem=False):
   
    reviews_text=list(map(lambda x:BeautifulSoup(x,'html.parser').get_text(),reviews))
    
    reviews_text=list(map(lambda x:re.sub("[^a-zA-Z]"," ",x),reviews_text))
    
    words=list(map(lambda x: x.lower().split(),reviews_text))
    
    if remove_stopwords:
        set_of_stopwords=set(stopwords.words("english"))
        meaningful_words=list(map(lambda x:[w for w in x if not w in set_of_stopwords],words))
    
    
    
    if stem:
        porter_stemmer = PorterStemmer()
        wordnet_lemmatizer = WordNetLemmatizer()
        stemmed_words = list(map(lambda x: [porter_stemmer.stem(w) for w in x], meaningful_words))
        stemmed_words = list(map(lambda x:[wordnet_lemmatizer.lemmatize(w) for w in x], stemmed_words))
        clean_review=map(lambda x:' '.join(x),stemmed_words)
    else:
        clean_review=list(map(lambda x:' '.join(x),meaningful_words))
    return clean_review

In [36]:
clean_test_reviews=clean_reviews(list(test['review'].values),remove_stopwords=True)
test_data_features=vectorizer.transform(clean_test_reviews)
test_data_features=test_data_features.toarray()

In [37]:
train_data_features.shape

(25000, 5000)

In [38]:
rf_clf= RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0) 
rf_clf_score=cross_val_score(rf_clf,train_data_features,train['sentiment'],cv=5,scoring='accuracy',n_jobs=-1).mean()

In [39]:
rf_clf_score

0.84268

In [40]:
rf_clf.fit(train_data_features,train['sentiment'])
result=rf_clf.predict(test_data_features)


In [42]:
out=pd.DataFrame(data={"id":test["id"],"sentiment":result})
out.to_csv('Bag_of_Words_rf_clf_result.csv',index=False,quoting=3)